# Evolution of red sequence on WaZP DP0 clusters

### Andressa Wille 

#### Universidade Tecnológica Federal do Paraná (UTFPR)
#### Laboratório Interinstitucional de e-Astronomia (LIneA)


## Objectives & Research Question:
* Study the red fraction evolution of members of WaZP clusters on DP0;
* Evaluate the performance of the WaZP algorithm, depending on input data;
* What is the impact of photo-z uncertainties on the red sequence evolution?  

## Background:
The Wavelet Z Photometric (WaZP) is an optical cluster finder that detects clusters using photometric redshift information. The algorithm can be used to select cluster members to study different aspects in cluster evolution, as the color-magnitude diagram feature named red sequence.

https://github.com/linea-it/wazp


## Methodology:

Imports

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd
from scipy.optimize import curve_fit
import sys
from scipy.interpolate import InterpolatedUnivariateSpline as spline
from scipy.integrate import quad
from scipy.special import erf
import sqlalchemy
from gavodb import DBManager

%matplotlib inline

### VAC QAs:

Processes on DES Science Portal:

| Pipeline | Product Log|
|---|:--:|
| Photo-z Training   | [7028](https://des-portal.linea.org.br/VP/getViewProcessCon?process_id=7028) |
| Photo-z Compute    | [7030](https://des-portal.linea.org.br/VP/getViewProcessCon?process_id=7030) |
| VAC DP0            | [7032](https://des-portal.linea.org.br/VP/getViewProcessCon?process_id=7032) |
| VAC DC2            | [7011](https://des-portal.linea.org.br/VP/getViewProcessCon?process_id=7011) |


In [ ]:
dbm = DBManager()

#### VAC DC2

In [ ]:
vac_dc2_tables = dbm.get_tablelist_from_pid('7011')

In [ ]:
vac_dc2_tables

In [ ]:
vac_dc2 = dbm.get_db_table('vac_cluster_simulation.footprint_7011', ['ra', 'dec'])

In [ ]:
vac_dataframe = pd.DataFrame(data=vac_dc2)
vac_dataframe = vac_dataframe.T
vac_dataframe.columns = ['ra', 'dec']
dc2 = vac_dataframe
dc2

#### VAC DP0

In [ ]:
vac_dp0_tables = dbm.get_tablelist_from_pid('7032')
vac_dp0_tables

In [ ]:
vac_dp0 = dbm.get_db_table('vac_cluster_simulation.catalog_7032', ['z_best'])

In [ ]:
vac_dp0 = pd.DataFrame(data=vac_dp0)
vac_dp0 = vac_dp0.T
vac_dp0.columns = ['z_best']
vac_dp0

#### N(z)

In [ ]:
bins = np.linspace(0,3,61)
plt.grid(True)
plt.hist(vac_dp0['z_best'], bins=bins, histtype='step', color="darkblue")
plt.xlabel("z")
plt.ylabel("Counts")
#plt.title("Redshift Distribution")
plt.xlim(0,3)
plt.tight_layout()
plt.savefig('photozbest.png')


### Data: 
   #### Catalogs
* Catalogs of WaZP cluster members in DC2 (truth mags, truth z) and DP0 ("observed" mags, photo-z DNF);
* Truth DC2 halo members catalog, used as reference.


In [ ]:
#DP0 catalog
wazp_dp0 = Table.read('/archive/user/aguena/cats/dp0/wazp/7038/wazp_membership.fits', format='fits')
#wazp_dp0
print(wazp_dp0.colnames)

In [ ]:
#DC2 catalog
wazp_dc2 = Table.read('/archive/user/aguena/cats/dc2/cosmoDC2_v1.1.4/extragal/full/wazp/6948/wazp_membership.fits', format='fits')
#wazp_dc2
print(wazp_dc2.colnames)

______________________________

The truth catalog is a matching between the WaZP clusters catalog and the halos catalog.
For this part of the work, we used ClEvaR, a library to validate cluster detection, to match the catalogs.

In [ ]:
from clevar import ClCatalog
from clevar.match import MembershipMatch

In [ ]:
# read catalogs
wazp = ClCatalog.read('/archive/user/aguena/cats/dc2/cosmoDC2_v1.1.4/extragal/full/wazp/6948/wazp_cluster.fits', 
                      'wazp', tags={'id':'id','ngals':'ngals','zp_bright':'zp_bright'}, full=False)
halo = ClCatalog.read('/archive/user/aguena/cats/dc2/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits', 
                      'halo', tags={'id':'halo_id'}, full=False)
mt = MembershipMatch()
# read matching info
mt.load_matches(wazp, halo, '/archive/user/aguena/notebooks_backup/wazp/dc2/6948/mem_pmem')

Select richness interval.

In [ ]:
mask_25_truth = (wazp['NGALS']>25)&(wazp['NGALS']<35)
mask_35_truth = (wazp['NGALS']>35)&(wazp['NGALS']<50)
mask_50_truth = (wazp['NGALS']>50)

richness_truth = [mask_25_truth, mask_35_truth, mask_50_truth]

Do the matching.

In [ ]:
from clevar.match import get_matched_pairs
for i, richness in enumerate(richness_truth):
    wazp_matched, halos_matched = get_matched_pairs(wazp, halo, 'cross', mask1=richness)
    if i == 0:
        halos_matched_25 = halos_matched
    if i == 1:
        halos_matched_35 = halos_matched
    if i == 2:
        halos_matched_50 = halos_matched
        
print(f'25<Ngals<35:{halos_matched_25}\n35<Ngals<50:{halos_matched_35}\nNgals>50:{halos_matched_50}')

Nice. We have the halos. Now we need the halo members.

In [ ]:
halo_members_path = '/archive/user/aguena/cats/dc2/cosmoDC2_v1.1.4/extragal/full/halos/'
halo_members = Table.read(halo_members_path + 'halos_m200c_13.0_members_nfwpmem.fits', format='fits')

In [ ]:
halo_members['halo_id'] = halo_members['halo_id'].astype(str)

In [ ]:
mem_mask = [HALO_ID in halos_matched_25.id_dict for HALO_ID in halo_members['halo_id']]
truth_25 = halo_members[mem_mask]
print(truth_25.colnames)
#truth_25

mem_mask = [HALO_ID in halos_matched_35.id_dict for HALO_ID in halo_members['halo_id']]
truth_35 = halo_members[mem_mask]
#truth_35

mem_mask = [HALO_ID in halos_matched_50.id_dict for HALO_ID in halo_members['halo_id']]
truth_50 = halo_members[mem_mask]
#truth_50

In [ ]:
truth_richness = [truth_25, truth_35, truth_50]

_______

#### Data selection

Select the catalog (wazp_dc2, wazp_dp0 or truth) and the magnitude bands to plot examples of CMDs and color histograms.

In [ ]:
catalog = wazp_dp0

mag1 = 'r'
mag2 = 'i'

Magnitude cut $m_{r}^{*}$

In [ ]:
data = np.loadtxt('/archive/user/aguena/cats/dc2/mstar/DC2_z_star.dat')
mstar = spline(*data[data[:,0]<2].T)

In [ ]:
mask_mag = (catalog['mag_z']<=mstar(catalog['ZP'])+1.5)

Select the richness intervals.

In [ ]:
mask_25 = (catalog['NGALS']>25)&(catalog['NGALS']<35)
mask_35 = (catalog['NGALS']>35)&(catalog['NGALS']<50)
mask_50 = (catalog['NGALS']>50)

richness = [mask_25, mask_35, mask_50]

### Redshift distribution

In [ ]:
plt.rcParams['font.size'] = 14

bins = np.linspace(0,1.5,16)
plt.hist(catalog['ZP'], bins=bins, ec="black", fc="mediumseagreen")
plt.xlabel("z")
plt.ylabel("Counts")
#plt.title("Redshift Distribution")
#plt.grid(True)
plt.tight_layout()
plt.savefig('z_dp0.png')

print(min(catalog['ZP']))
print(max(catalog['ZP']))

### Spacial distribution
Let's compare DP0 and DC2 spacial distribution.

In [ ]:
plt.rcParams['font.size'] = 10

fig, (ax1, ax2) = plt.subplots(1, 2)
#fig.suptitle('        Spacial distribution of DC2 and DP0')

ra = wazp_dc2['RA']
ra[ra >180.0] -= 360
dec = wazp_dc2['DEC']

im1 = ax1.hexbin(ra, dec, None,  mincnt=1, cmap='summer', gridsize=[200,100])
ax1.set_title('DC2 catalog')

ra2 = wazp_dp0['RA']
ra2[ra2 >180.0] -= 360
dec2 = wazp_dp0['DEC']

im2 = ax2.hexbin(ra2, dec2, None,  mincnt=1, cmap='summer', gridsize=[200,100])
ax2.set_title('DP0 catalog')

for ax in fig.get_axes():
    ax.set_xlabel("R.A. (deg)")  
    ax.set_ylabel("Dec. (deg)")
    ax.set_xlim(43,80)
    ax.set_ylim(-50,-20)
    ax.label_outer()

#fig.subplots_adjust(right=0.9)
cax = fig.add_axes([1.0, 0.15, 0.025, 0.66])
cbar = fig.colorbar(im1, cax=cax, cmap='summer', orientation='vertical')

plt.tight_layout()
fig.savefig('ra_x_dec_dp0.png')

### Color magnitude diagrams

Select the redshift interval to plot CMDs of the selected catalog.
The name of the columns can vary according the catalog.

In [ ]:
plt.rcParams['font.size'] = 14

zpmax = 0.6
zpmin = 0.5

Nbins = 1
dz = (zpmax - zpmin)/Nbins

for j in range(0, Nbins):
    z1 = zpmin + (j * dz)
    z2 = z1 + dz
    round(z1,1)
    round(z2,1)
    #print(z1, z2)
    
    for i, richness_bins in enumerate(richness):

        mask_z = (catalog['ZP']>z1)&(catalog['ZP']<z2)
        mask = (richness_bins)&(mask_z)&(mask_mag) 

        plt.figure()#, dpi=300)
        catalog_color = catalog[f'mag_{mag1}'][mask] - catalog[f'mag_{mag2}'][mask]
        color = np.array(catalog_color)
        mag = np.array(catalog[f'mag_{mag2}'][mask])
        pmem = np.array(catalog['PMEM'][mask])
        plt.hist2d(mag, color, bins=70, weights=pmem, cmin=0.1, cmax=None, cmap='viridis')
        cbar = pl.colorbar(label="density")

        plt.xlim(18.5, 22.6)
        plt.ylim(-0.1, 1.6)
        plt.clim(2.0, 30.0)
        
        if i == 0:
             text = '25<$N_{gals}$<35'
        if i == 1:
             text='35<$N_{gals}$<50'
        if i == 2:
            text='$N_{gals}$>50'
            
        #plt.title(f'Color-magnitude diagram \nWaZP members in DP0 ({round(z1,1)}<$z$<{round(z2,1)}, {text})', fontsize = 13)
        plt.text(19, 1.45, text, fontsize=13,
                 bbox=dict(facecolor='white', alpha=0.6, boxstyle="round"))
        plt.xlabel('mag '+mag2)
        plt.ylabel(f'color {mag1}-{mag2}')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'cmd_dp0_{i}.png')

### Color histograms (DC2 and DP0)
* After selecting the richness range and the magnitudes, we have to separate the data in redshift bins.
* The red and blue populations were characterized by Gaussian distributions on color histograms. We chose the colors that envelop the 4000 Å break. The 4000 Å break is caused by the absorption of high energy radiation from metals in stellar atmospheres and by a deficiency of hot, blue stars. That implies the presence of old stellar populations.
    We have the following distribution of colors: $0.1 < z < 0.2$ in g-r, $0.3 < z < 0.6$ in r-i, $0.7 < z < 1.0$ in i-z, $1.1 < z < 1.4$ in z-y.

* We integrated the curves to find the number of red and total members and calculate the red fraction in each redshift bin.

Define gaussian fit and integral.

Gaussian params: A, $\mu$ and $\sigma$ are the height, the mean and the standard deviation, respectively.

In [ ]:
def gauss(x, mu, sigma, A):
    return A*np.exp(-(x-mu)**2/2/sigma**2)

def bimodal(x, mu1, sigma1, A1, mu2, sigma2, A2):
    return gauss(x,mu1,sigma1,A1)+gauss(x,mu2,sigma2,A2)

In [ ]:
def gauss_integral(y):
    return A2*np.exp(-(y-mu2)**2/2/sigma2**2)
def gauss2_integral(y):
    return (A1*np.exp(-(y-mu1)**2/2/sigma1**2)+A2*np.exp(-(y-mu2)**2/2/sigma2**2))

Define error propagation.

This propagation was made according to the general equation:
$err_{w}^2 = \left(\frac{\partial w}{\partial x}\right)^2 err_{x}^2 + \left(\frac{\partial w}{\partial y}\right)^2 err_{y}^2 + \left(\frac{\partial w}{\partial z}\right)^2 err_{z}^2 + ...$


erro1 -> $\left(\frac{\partial w}{\partial x}\right)^2 err_{x}^2 = \left(\frac{\partial}{\partial A}  \int_{x_{min}}^{x_{max}} A e^{\frac{-(x-\mu)^2}{2\sigma ^2}}\right)^2 err_{A}^2 =\left(\sqrt{\frac{\pi}{2}}\, \sigma \, erf\left(\frac{x-\mu}{\sigma \sqrt{2}}\right)\right)^2 err_{A}^2$


erro2 -> $\left(\frac{\partial w}{\partial y}\right)^2 err_{y}^2 = \left(\frac{\partial}{\partial \mu}  \int_{x_{min}}^{x_{max}} A e^{\frac{-(x-\mu)^2}{2\sigma ^2}}\right)^2 err_{\mu}^2 = \left(-A e^{\frac{-(\mu-x)^2}{2\sigma ^2}}\right)^2 \, err_{\mu}^2$

erro3 -> $\left(\frac{\partial w}{\partial z}\right)^2 err_{z}^2 =
     \left(\frac{\partial}{\partial \sigma}  \int_{x_{min}}^{x_{max}} A e^{\frac{-(x-\mu)^2}{2\sigma ^2}}\right)^2 err_{\sigma}^2 = \left(\frac{A(\mu - x) e^{\frac{-(x-\mu)^2}{2\sigma ^2}}}{\sigma} - \sqrt{\frac{\pi}{2}} \, A \, erf\left(\frac{\mu - x}{\sigma \sqrt{2}}\right)\right)^2 err_{\sigma}^2$
     


In [ ]:
def error(mu1, sigma1, A1, mu2, sigma2, A2, mu1_err, sigma1_err, A1_err, mu2_err, sigma2_err, A2_err):
    erro1 = ((np.sqrt(np.pi/2))*sigma2*erf((mu2-x.max())/(sigma2*np.sqrt(2)))) - ((np.sqrt(np.pi/2))*sigma2*erf((mu2-x.min())/(sigma2*np.sqrt(2))))
    erro1 = (erro1**2) * (A2_err)**2

    erro2 = (-A2* np.exp(-(mu2-x.max())**2/2/sigma2**2)) - (-A2* np.exp(-(mu2-x.min())**2/2/sigma2**2))
    erro2 = (erro2**2) * (mu2_err)**2

    erro3_1 = ((A2*(mu2-x.max()* np.exp(-(mu2-x.max())**2/2/sigma2**2)))/(sigma2) -A2*np.sqrt(np.pi/2)*erf((mu2-x.max())/(sigma2*np.sqrt(2))))
    erro3_2 = ((A2*(mu2-x.min()* np.exp(-(mu2-x.min())**2/2/sigma2**2)))/(sigma2) -A2*np.sqrt(np.pi/2)*erf((mu2-x.min())/(sigma2*np.sqrt(2))))
    erro3 = erro3_1 - erro3_2
    erro3 = (erro3**2) * (sigma2_err)**2

    #erro gaussiana vermelha
    erro_r = np.sqrt(erro1+erro2+erro3)
    #print(erro_r)

    erro4 = ((np.sqrt(np.pi/2))*sigma1*erf((mu1-x.max())/(sigma1*np.sqrt(2)))) - ((np.sqrt(np.pi/2))*sigma1*erf((mu1-x.min())/(sigma1*np.sqrt(2))))
    erro4 = (erro4**2) * (A1_err)**2

    erro5 = (-A1* np.exp(-(mu1-x.max())**2/2/sigma1**2)) - (-A1* np.exp(-(mu1-x.min())**2/2/sigma1**2))
    erro5 = (erro5**2) * (mu1_err)**2

    erro6_1 = ((A1*(mu1-x.max()* np.exp(-(mu1-x.max())**2/2/sigma1**2)))/(sigma1) -A1*np.sqrt(np.pi/2)*erf((mu1-x.max())/(sigma1*np.sqrt(2))))
    erro6_2 = ((A1*(mu1-x.min()* np.exp(-(mu1-x.min())**2/2/sigma1**2)))/(sigma1) -A1*np.sqrt(np.pi/2)*erf((mu1-x.min())/(sigma1*np.sqrt(2))))
    erro6 = erro6_1 - erro6_2
    erro6 = (erro6**2) * (sigma1_err)**2

    #erro gaussiana azul
    erro_b = np.sqrt(erro4+erro4+erro6)
    #print(erro_b)

    #erro gaussiana total
    erro = np.sqrt(erro1+erro2+erro3+erro4+erro4+erro6)
    #print(erro)

    return erro_r, erro

Select redshift interval for the example.

In [ ]:
z1 = 0.5
z2 = 0.6

Example:

In [ ]:
catalog_color = catalog[f'mag_{mag1}'] - catalog[f'mag_{mag2}']
mask_z = (catalog['ZP']>z1)&(catalog['ZP']<z2)

for i, richness_bins in enumerate(richness): 
    
    mask = (richness_bins)&(mask_z)&(mask_mag) 
    
    color = np.array(catalog_color[mask])
    pmem = np.array(catalog['PMEM'][mask])

    plt.figure(figsize=[7,5])
    y,x,_=plt.hist(color, bins=30, weights=pmem, fc="moccasin", ec='sandybrown', label='color distribution')
    x=(x[1:]+x[:-1])/2

    expected = (0.6, 0.2, 1000.0, 0.9, 0.1, 3000.0)
    params, cov = curve_fit(bimodal, x, y, expected)
    sigma=np.sqrt(np.diag(cov))
    x_fit = np.linspace(x.min(), x.max(), 500)

    plt.plot(x_fit, bimodal(x_fit, *params), color='black', lw=3, label='total')
    plt.plot(x_fit, gauss(x_fit, *params[0:3]), color='blue', lw=2, ls="--", label='blue population')
    plt.plot(x_fit, gauss(x_fit, *params[3:6]), color='red', lw=2, ls="--", label='red population')

    if i == 0:
        text = '25<$N_{gals}$<35'
    if i == 1:
        text='35<$N_{gals}$<50'
    if i == 2:
        text='$N_{gals}$>50'
            
    #plt.title(f'Color histogram \nWaZP members in DP0 ({round(z1,1)}<$z$<{round(z2,1)}, {text})', fontsize=14)
    plt.text(1.2, 1300, text, fontsize=13,
                 bbox=dict(facecolor='white', alpha=0.6, boxstyle="round"))
    plt.legend(loc='upper left', fontsize=11)
    plt.ylabel('number of galaxies')
    plt.xlim(0.0, 1.6)
    plt.ylim(0.0, 1450)
    plt.xlabel(f'{mag1}-{mag2}')
    plt.grid(True)
    data = pd.DataFrame(data={'params': params, 'err': sigma}, index=bimodal.__code__.co_varnames[1:])
    print(data)

    plt.savefig(f'color_hist_{i}.png')
    plt.show() 

For other redshift bins (and other colors), repeat the process below.

Select color.

In [ ]:
mag1 = 'g'
mag2 = 'r'

Select the richness bin.

In [ ]:
richness_bin = mask_50

Here you should put a csv file with the expected params for the Gaussian fit. Remember to look for the file with information for the catalog, richness and color that you choose.

In [ ]:
arquivo = pd.read_csv('expected_params/dp0_50_all.csv')

In [ ]:
arquivo

Arrays to save fractions and error.

Don't forget to re-run the cell with the arrays when you change the catalog or the richness.

In [ ]:
fraction = []
fraction_err = []
photozs = []

In [ ]:
plt.rcParams['font.size'] = 14

zpmax = 1.4
zpmin = 0.1

dz = 0.1

for z1 in np.arange(zpmin, zpmax, dz):
    
    z1 = round(z1,1)
    z2 = round(z1+dz,1)
    photoz = (z1+z2)/2
    
    #_______________________#
    #use this part for the all colors analysis
    if z1 <= 0.2 :
        mag1 = 'g'
        mag2 = 'r'
    if z1 > 0.2 and z1 <= 0.6:
        mag1 = 'r'
        mag2 = 'i'
    if z1 > 0.6 and z1 <= 1.0:
        mag1 = 'i'
        mag2 = 'z'
    if z1 > 1.0 and z1 <= 1.3:
        mag1 = 'z'
        mag2 = 'y'
    #_______________________#
    
    catalog_color = catalog[f'mag_{mag1}'] - catalog[f'mag_{mag2}']

    mask_z = (catalog['ZP']>z1)&(catalog['ZP']<z2)
    mask = (richness_bin)&(mask_z)&(mask_mag) 

    color = np.array(catalog_color[mask])
    pmem = np.array(catalog['PMEM'][mask])

    plt.figure(figsize=[7,5])
    y,x,_=plt.hist(color, bins=30, weights=pmem, fc="moccasin", ec='sandybrown', label='color distribution')
    x=(x[1:]+x[:-1])/2
    
    mask_params = (arquivo['z']==z1)

    mu1 = float(arquivo['mu1'][mask_params])
    sigma1 = float(arquivo['sigma1'][mask_params])
    A1 = float(arquivo['A1'][mask_params])
    mu2 = float(arquivo['mu2'][mask_params])
    sigma2 = float(arquivo['sigma2'][mask_params])
    A2 = float(arquivo['A2'][mask_params])
    
    expected = (mu1, sigma1, A1, mu2, sigma2, A2)
    params, cov = curve_fit(bimodal, x, y, expected)
    sigma=np.sqrt(np.diag(cov))
    x_fit = np.linspace(x.min(), x.max(), 500)

    plt.plot(x_fit, bimodal(x_fit, *params), color='black', lw=3, label='total')
    plt.plot(x_fit, gauss(x_fit, *params[0:3]), color='blue', lw=2, ls="--", label='blue population')
    plt.plot(x_fit, gauss(x_fit, *params[3:6]), color='red', lw=2, ls="--", label='red population')
    plt.title(f'Color histogram \nWaZP members in DC2 ({round(z1,1)}<$z$<{round(z2,1)})', fontsize=14)
    
    #text = '25<$N_{gals}$<35'
    #plt.text(-0.4, 2250, text, fontsize=13,bbox=dict(facecolor='white', alpha=0.6, boxstyle="round"))
    
    plt.legend(fontsize=11)
    plt.ylabel('number of galaxies')
    plt.xlabel(f'{mag1}-{mag2}')
    plt.grid(True)
    data = pd.DataFrame(data={'params': params, 'err': sigma}, index=bimodal.__code__.co_varnames[1:])
    print(data)
    plt.show() 

    mu1 = data.to_numpy()[0][0]
    sigma1 = data.to_numpy()[1][0]
    A1 = data.to_numpy()[2][0]
    mu2 = data.to_numpy()[3][0]
    sigma2 = data.to_numpy()[4][0]
    A2 = data.to_numpy()[5][0]

    mu1_err = data.to_numpy()[0][1]
    sigma1_err = data.to_numpy()[1][1]
    A1_err = data.to_numpy()[2][1]
    mu2_err = data.to_numpy()[3][1]
    sigma2_err = data.to_numpy()[4][1]
    A2_err = data.to_numpy()[5][1]

    erro_r, erro = error(mu1, sigma1, A1, mu2, sigma2, A2, mu1_err, sigma1_err, A1_err, mu2_err, sigma2_err, A2_err)
    
    print(f'{round(z1,1)}<z<{round(z2,1)}')
    n_red_galaxies, n_red_galaxies_err = quad(gauss_integral, x.min(), x.max())
    print(f'number of red galaxies:{n_red_galaxies} +- {erro_r}')
    n_galaxies, n_galaxies_err = quad(gauss2_integral, x.min(), x.max())
    print(f'number of galaxies:{n_galaxies} +- {erro}')
    red_fraction = n_red_galaxies/n_galaxies
    red_fraction_err = (n_red_galaxies/n_galaxies)* np.sqrt((erro_r/n_red_galaxies)**2+(erro/n_galaxies)**2)
    print(f'red fraction:{red_fraction} +- {red_fraction_err}')
    
    fraction.append(red_fraction)
    fraction_err.append(red_fraction_err)
    photozs.append(photoz)
    
out = np.transpose([photozs, fraction, fraction_err])
name = 'dp0_50_all.txt' #change the name of the catalog.
np.savetxt(name, out, header='redshift   fractions   errors', fmt='%1.6e')

### Color histograms (truth catalog)

The color histogram of the truth catalog is a bit different, because the red population in this case is the Gaussian that characterizes the "is on red sequence" column. 

Define gaussian integral and error propagation.

In [ ]:
def gauss_integral_red(y):
    return A*np.exp(-(y-mu)**2/2/sigma**2)

In [ ]:
def error_truth_total(mu1, sigma1, A1, mu2, sigma2, A2, mu1_err, sigma1_err, A1_err, mu2_err, sigma2_err, A2_err):
    erro1 = ((np.sqrt(np.pi/2))*sigma2*erf((mu2-x.max())/(sigma2*np.sqrt(2)))) - ((np.sqrt(np.pi/2))*sigma2*erf((mu2-x.min())/(sigma2*np.sqrt(2))))
    erro1 = (erro1**2) * (A2_err)**2

    erro2 = (-A2* np.exp(-(mu2-x.max())**2/2/sigma2**2)) - (-A2* np.exp(-(mu2-x.min())**2/2/sigma2**2))
    erro2 = (erro2**2) * (mu2_err)**2

    erro3_1 = ((A2*(mu2-x.max()* np.exp(-(mu2-x.max())**2/2/sigma2**2)))/(sigma2) -A2*np.sqrt(np.pi/2)*erf((mu2-x.max())/(sigma2*np.sqrt(2))))
    erro3_2 = ((A2*(mu2-x.min()* np.exp(-(mu2-x.min())**2/2/sigma2**2)))/(sigma2) -A2*np.sqrt(np.pi/2)*erf((mu2-x.min())/(sigma2*np.sqrt(2))))
    erro3 = erro3_1 - erro3_2
    erro3 = (erro3**2) * (sigma2_err)**2

    erro4 = ((np.sqrt(np.pi/2))*sigma1*erf((mu1-x.max())/(sigma1*np.sqrt(2)))) - ((np.sqrt(np.pi/2))*sigma1*erf((mu1-x.min())/(sigma1*np.sqrt(2))))
    erro4 = (erro4**2) * (A1_err)**2

    erro5 = (-A1* np.exp(-(mu1-x.max())**2/2/sigma1**2)) - (-A1* np.exp(-(mu1-x.min())**2/2/sigma1**2))
    erro5 = (erro5**2) * (mu1_err)**2

    erro6_1 = ((A1*(mu1-x.max()* np.exp(-(mu1-x.max())**2/2/sigma1**2)))/(sigma1) -A1*np.sqrt(np.pi/2)*erf((mu1-x.max())/(sigma1*np.sqrt(2))))
    erro6_2 = ((A1*(mu1-x.min()* np.exp(-(mu1-x.min())**2/2/sigma1**2)))/(sigma1) -A1*np.sqrt(np.pi/2)*erf((mu1-x.min())/(sigma1*np.sqrt(2))))
    erro6 = erro6_1 - erro6_2
    erro6 = (erro6**2) * (sigma1_err)**2

    #erro gaussiana total
    erro = np.sqrt(erro1+erro2+erro3+erro4+erro4+erro6)
    #print(erro)

    return erro

In [ ]:
def error_truth(mu, sigma, A):
    
    erro7 = ((np.sqrt(np.pi/2))*sigma*erf((mu-x.max())/(sigma*np.sqrt(2)))) - ((np.sqrt(np.pi/2))*sigma*erf((mu-x.min())/(sigma*np.sqrt(2))))
    erro7 = (erro7**2) * (A_err)**2

    erro8 = (-A* np.exp(-(mu-x.max())**2/2/sigma**2)) - (-A* np.exp(-(mu-x.min())**2/2/sigma**2))
    erro8 = (erro8**2) * (mu_err)**2

    erro9_1 = ((A*(mu-x.max()* np.exp(-(mu-x.max())**2/2/sigma**2)))/(sigma) -A*np.sqrt(np.pi/2)*erf((mu-x.max())/(sigma*np.sqrt(2))))
    erro9_2 = ((A*(mu-x.min()* np.exp(-(mu-x.min())**2/2/sigma**2)))/(sigma) -A*np.sqrt(np.pi/2)*erf((mu-x.min())/(sigma*np.sqrt(2))))
    erro9 = erro9_1 - erro9_2
    erro9 = (erro9**2) * (sigma_err)**2
    
    #erro gaussiana vermelha truth
    erro_r = np.sqrt(erro7+erro8+erro9)
    #print(erro_r)

    return erro_r

Again, first let's see some examples.

In [ ]:
mag1 = 'r'
mag2 = 'i'
z1 = 1.3
z2 = 1.4

In [ ]:
for i, truth in enumerate(truth_richness): 

    mask_z = (truth['redshift_true']>z1)&(truth['redshift_true']<z2)
    mask_isred = (truth[f'is_on_red_sequence_{mag1}{mag2}']==True)
    mask = (mask_z)
    mask2 = (mask_z)&(mask_isred)

    truth_color = truth[f'mag_true_{mag1}'][mask] - truth[f'mag_true_{mag2}'][mask]
    color = np.array(truth_color)
    pmem = np.array(truth['pmem_nfw2d'][mask])

    truth_color2 = truth[f'mag_true_{mag1}'][mask2] - truth[f'mag_true_{mag2}'][mask2]
    color2 = np.array(truth_color2)
    pmem2 = np.array(truth['pmem_nfw2d'][mask2])

    plt.figure(figsize=[8,5])
    y,x,_=plt.hist(color, bins=20, weights=pmem, fc="moccasin", ec='sandybrown')
    x=(x[1:]+x[:-1])/2
    expected = (0.25, 0.2, 20.0, 1.4, 0.2, 400.0)
    params, cov = curve_fit(bimodal, x, y, expected)
    sigma=np.sqrt(np.diag(cov))
    x_fit = np.linspace(x.min(), x.max(), 500)


    y2,x2,_2=plt.hist(color2, bins=20, weights=pmem2, fc='floralwhite', ec='sandybrown')
    x2=(x2[1:]+x2[:-1])/2
    expected2 = (1.0, 0.2, 4000.0)
    params2, cov2 = curve_fit(gauss, x2, y2, expected2)
    sigma2=np.sqrt(np.diag(cov2))
    x_fit2 = np.linspace(x2.min(), x2.max(), 500)
    
    if i == 0:
        text ='25<$N_{gals}$<35'
    if i == 1:
        text='35<$N_{gals}$<50'
    if i == 2:
        text='$N_{gals}$>50'

    plt.plot(x_fit, bimodal(x_fit, *params), color='black', lw=3, label='all members matched')
    plt.plot(x_fit2, gauss(x_fit2, *params2), color='red', lw=3, label=f'is_on_red_sequence_{mag1}{mag2}')
    plt.title(f'Color histogram \nTruth catalog ({round(z1,1)}<$z$<{round(z2,1)}, {text})', fontsize=14)
    plt.text(-0.1, 900, text, fontsize=13, bbox=dict(facecolor='white', alpha=0.6, boxstyle="round"))
    plt.legend(fontsize=11)
    plt.ylabel('number of galaxies')
    plt.xlabel(f'{mag1}-{mag2}')
    #plt.xlim(0.0, 1.6)
    #plt.ylim(0.0, 2500)
    plt.grid(True)

    plt.savefig(f'color_hist_halos_{i}.png')
    plt.show() 

    data = pd.DataFrame(data={'params': params, 'err': sigma}, index=bimodal.__code__.co_varnames[1:])
    print(data)

    mu1 = data.to_numpy()[0][0]
    sigma1 = data.to_numpy()[1][0]
    A1 = data.to_numpy()[2][0]
    mu2 = data.to_numpy()[3][0]
    sigma2 = data.to_numpy()[4][0]
    A2 = data.to_numpy()[5][0]

    mu1_err = data.to_numpy()[0][1]
    sigma1_err = data.to_numpy()[1][1]
    A1_err = data.to_numpy()[2][1]
    mu2_err = data.to_numpy()[3][1]
    sigma2_err = data.to_numpy()[4][1]
    A2_err = data.to_numpy()[5][1]

    data2 = pd.DataFrame(data={'params red': params2, 'err red': sigma2}, index=gauss.__code__.co_varnames[1:])
    print(data2)

    mu = data.to_numpy()[0][0]
    sigma = data.to_numpy()[1][0]
    A = data.to_numpy()[2][0]

    mu_err = data.to_numpy()[0][1]
    sigma_err = data.to_numpy()[1][1]
    A_err = data.to_numpy()[2][1]

For the complete analysis of truth catalog, follow the instructions below.

Select the color (is_on_red_sequence is available only for g-r and r-i)

In [ ]:
mag1 = 'r'
mag2 = 'i'

Select the richness.

In [ ]:
truth = truth_50

Change the csv file with the expected params for the gaussian fit.

In [ ]:
arquivo = pd.read_csv('truth_50_all.csv')

In [ ]:
arquivo

Arrays to save fractions and errors.

In [ ]:
fractions = []
fractions_err = []
photozs = []

Total fit

In [ ]:
total = []
total_err = []

In [ ]:
plt.rcParams['font.size'] = 14

zpmax = 0.7
zpmin = 0.1

dz = 0.1

for z1 in np.arange(zpmin, zpmax, dz):
    
    z1 = round(z1,1)
    z2 = round(z1+dz,1)
    photoz = (z1+z2)/2
    
    #_______________________#
    #use this part for the all colors analysis and change zpmax = 0.7.
    if z1 <= 0.2 :
        mag1 = 'g'
        mag2 = 'r'
    if z1 > 0.2 and z1 <= 0.6:
        mag1 = 'r'
        mag2 = 'i'
    #_______________________#
    
    catalog_color = truth[f'mag_true_{mag1}'] - truth[f'mag_true_{mag2}']

    mask_z = (truth['redshift_true']>z1)&(truth['redshift_true']<z2)
    mask = (mask_z) 

    color = np.array(catalog_color[mask])
    pmem = np.array(truth['pmem_nfw2d'][mask])

    plt.figure(figsize=[7,5])
    y,x,_=plt.hist(color, bins=30, weights=pmem, fc="moccasin", ec='sandybrown', label='color distribution')
    x=(x[1:]+x[:-1])/2
    
    mask_params = (arquivo['z']==z1)

    mu1 = float(arquivo['mu1'][mask_params])
    sigma1 = float(arquivo['sigma1'][mask_params])
    A1 = float(arquivo['A1'][mask_params])
    mu2 = float(arquivo['mu2'][mask_params])
    sigma2 = float(arquivo['sigma2'][mask_params])
    A2 = float(arquivo['A2'][mask_params])
    
    expected = (mu1, sigma1, A1, mu2, sigma2, A2)
    params, cov = curve_fit(bimodal, x, y, expected)
    sigma=np.sqrt(np.diag(cov))
    x_fit = np.linspace(x.min(), x.max(), 500)

    plt.plot(x_fit, bimodal(x_fit, *params), color='black', lw=3, label='total')
    #plt.plot(x_fit, gauss(x_fit, *params[0:3]), color='blue', lw=2, ls="--", label='blue population')
    #plt.plot(x_fit, gauss(x_fit, *params[3:6]), color='red', lw=2, ls="--", label='red population')
    plt.title(f'Color histogram \nWaZP members in truth catalog ({round(z1,1)}<$z$<{round(z2,1)})', fontsize=14)
    
    #text = '25<$N_{gals}$<35'
    #plt.text(-0.4, 2250, text, fontsize=13,bbox=dict(facecolor='white', alpha=0.6, boxstyle="round"))
    
    plt.legend(fontsize=11)
    plt.ylabel('number of galaxies')
    plt.xlabel(f'{mag1}-{mag2}')
    plt.grid(True)
    data = pd.DataFrame(data={'params': params, 'err': sigma}, index=bimodal.__code__.co_varnames[1:])
    print(data)
    plt.show() 

    mu1 = data.to_numpy()[0][0]
    sigma1 = data.to_numpy()[1][0]
    A1 = data.to_numpy()[2][0]
    mu2 = data.to_numpy()[3][0]
    sigma2 = data.to_numpy()[4][0]
    A2 = data.to_numpy()[5][0]

    mu1_err = data.to_numpy()[0][1]
    sigma1_err = data.to_numpy()[1][1]
    A1_err = data.to_numpy()[2][1]
    mu2_err = data.to_numpy()[3][1]
    sigma2_err = data.to_numpy()[4][1]
    A2_err = data.to_numpy()[5][1]

    erro = error_truth_total(mu1, sigma1, A1, mu2, sigma2, A2, mu1_err, sigma1_err, A1_err, mu2_err, sigma2_err, A2_err)
    
    print(f'{round(z1,1)}<z<{round(z2,1)}')
    n_galaxies, n_galaxies_err = quad(gauss2_integral, x.min(), x.max())
    print(f'number of galaxies:{n_galaxies} +- {erro}')
    
    total.append(n_galaxies)
    total_err.append(erro)
    photozs.append(photoz)

In [ ]:
total, total_err

"Is on red sequence" fit

In [ ]:
red = []
red_err = []

In [ ]:
plt.rcParams['font.size'] = 14

zpmax = 0.7
zpmin = 0.1

dz = 0.1

for z1 in np.arange(zpmin, zpmax, dz):
    
    z1 = round(z1,1)
    z2 = round(z1+dz,1)
    
    #_______________________#
    #use this part for the all colors analysis and change zpmax = 0.7.
    if z1 <= 0.2 :
        mag1 = 'g'
        mag2 = 'r'
    if z1 > 0.2 and z1 <= 0.6:
        mag1 = 'r'
        mag2 = 'i'
    #_______________________#
    
    catalog_color = truth[f'mag_true_{mag1}'] - truth[f'mag_true_{mag2}']

    mask_z = (truth['redshift_true']>z1)&(truth['redshift_true']<z2)
    mask_isred = (truth[f'is_on_red_sequence_{mag1}{mag2}']==True)
    mask2 = (mask_z)&(mask_isred)

    color = np.array(catalog_color[mask2])
    pmem = np.array(truth['pmem_nfw2d'][mask2])

    plt.figure(figsize=[7,5])
    y,x,_=plt.hist(color, bins=30, weights=pmem, fc="moccasin", ec='sandybrown', label='color distribution')
    x=(x[1:]+x[:-1])/2
    
    mask_params = (arquivo['z']==z1)

    mu = float(arquivo['mu2'][mask_params])
    sigma = float(arquivo['sigma2'][mask_params])
    A = float(arquivo['A2'][mask_params])
    
    expected = (mu, sigma, A)
    params, cov = curve_fit(gauss, x, y, expected)
    sigma=np.sqrt(np.diag(cov))
    x_fit = np.linspace(x.min(), x.max(), 500)

    plt.plot(x_fit, gauss(x_fit, *params), color='red', lw=3, label='is on red sequence')
    plt.title(f'Color histogram \nWaZP members in truth catalog ({round(z1,1)}<$z$<{round(z2,1)})', fontsize=14)
    
    #text = '25<$N_{gals}$<35'
    #plt.text(-0.4, 2250, text, fontsize=13,bbox=dict(facecolor='white', alpha=0.6, boxstyle="round"))
    
    plt.legend(fontsize=11)
    plt.ylabel('number of galaxies')
    plt.xlabel(f'{mag1}-{mag2}')
    plt.grid(True)
    data = pd.DataFrame(data={'params red': params, 'err red': sigma}, index=gauss.__code__.co_varnames[1:])
    print(data)
    plt.show() 
    
    mu = data.to_numpy()[0][0]
    sigma = data.to_numpy()[1][0]
    A = data.to_numpy()[2][0]

    mu_err = data.to_numpy()[0][1]
    sigma_err = data.to_numpy()[1][1]
    A_err = data.to_numpy()[2][1]

    erro_r = error_truth(mu, sigma, A)
    
    print(f'{round(z1,1)}<z<{round(z2,1)}')
    n_red_galaxies, n_red_xies_err = quad(gauss_integral_red, x.min(), x.max())
    print(f'number of galaxies:{n_red_galaxies} +- {erro_r}')
    
    red.append(n_red_galaxies)
    red_err.append(erro_r)
    

In [ ]:
red, red_err

Calculate fractions and errors.

In [ ]:
for i in range(0, 6, 1):
    fraction = red[i]/total[i]
    fractions.append(fraction)
    fraction_err = (red[i]/total[i])* np.sqrt((red_err[i]/red[i])**2+(total_err[i]/total[i])**2)
    fractions_err.append(fraction_err)

In [ ]:
fractions

In [ ]:
fractions_err

In [ ]:
photozs

In [ ]:
out = np.transpose([photozs, fractions, fractions_err])
name = 'truth_50_all.txt' #change the name of the catalog.
np.savetxt(name, out, header='redshift   fractions   errors', fmt='%1.6e')

## Results:

Plot figures to see the evolution of red fraction with redshift for the sample analyzed.


In [ ]:
rich = ['25', '35', '50']
colors = ['ri', 'all']

In [ ]:
for i in rich:
    for j in colors:
        
        doc = f'dp0_{i}_{j}.txt'
        redshifts1, fractions1, fractions_err1= np.loadtxt(doc, usecols=(0,1,2), unpack=True)
        doc2 = f'dc2_{i}_{j}.txt'
        redshifts2, fractions2, fractions_err2= np.loadtxt(doc2, usecols=(0,1,2), unpack=True)
        doc3 = f'truth_{i}_{j}.txt'
        redshifts3, fractions3, fractions_err3= np.loadtxt(doc3, usecols=(0,1,2), unpack=True)

        plt.rcParams['font.size'] = 15

        plt.figure(figsize=[6,6], dpi=300)

        plt.plot(redshifts1, fractions1, 'o', color='green', label='WaZP in DP0')
        plt.errorbar(redshifts1, fractions1, yerr=fractions_err1, color='green', fmt='.')
        plt.plot(redshifts2, fractions2, 's', color='orangered', label='WaZP in DC2')
        plt.errorbar(redshifts2, fractions2, yerr=fractions_err2, color='orangered', fmt='.')
        plt.plot(redshifts3, fractions3, '^', color='teal', label='Truth catalog')
        plt.errorbar(redshifts3, fractions3, yerr=fractions_err3, color='teal', fmt='.')
        
        if j == 'all':
            plt.vlines(0.3, -0.2, 1.3, linestyles='dotted', color='black')
            plt.vlines(0.7, -0.2, 1.3, linestyles='dotted', color='black')
            plt.vlines(1.1, -0.2, 1.3, linestyles='dotted', color='black')
            plt.text(0.2, 0.0, 'g-r', fontsize=12)
            plt.text(0.5, 0.0, 'r-i', fontsize=12)
            plt.text(0.9, 0.0, 'i-z', fontsize=12)
            plt.text(1.23, 0.0, 'z-y', fontsize=12)
            
        plt.xlabel("redshift")
        plt.ylabel("red fraction")
        plt.ylim(-0.1, 1.2)
        plt.xlim(0.1, 1.4)
        plt.xticks([0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3])
        plt.yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
        
        if i == '25':
            text ='25<$N_{gals}$<35'
        if i == '35':
            text='35<$N_{gals}$<50'
        if i == '50':
            text='$N_{gals}$>50'
            
        if i == '25' and j == 'ri':
            text ='25<$N_{gals}$<35\n color r-i'
        if i == '35'and j == 'ri':
            text='35<$N_{gals}$<50\n color r-i'
        if i == '50' and j == 'ri':
            text='$N_{gals}$>50\n color r-i'
        
        plt.text(0.15, 1.05, text, fontsize=12, bbox=dict(facecolor='white', alpha=0.6, boxstyle="round"))
        #plt.title(f"Red fraction evolution {i}, {j}", fontsize=14)
        plt.legend(fontsize=12)
        plt.tight_layout()
        plt.savefig(f'frac_evol_{i}_{j}.png')


## Discussion and Conclusions:
* The first and last redshift bins have larger error bars because fewer objects are selected and the gaussian fit is not so precise.
* The red fraction evolution observed in wazp cluster members recovers the overall trend of the galaxies marked as "is on red sequence" in the truth catalog for color r-iin intermediate redshifts. The "truth" catalog results are less reliable at the most extreme redshifts because of the extrapolation of the redshift range in which the r-i band is valid. It seems necessary to separate the sample by considering the colors that envelope the 4000 A break.
* It is noticeable that the results of the three catalogs with richness $N_{gals} > 50$ are more concordant: the higher the richness, the better the capability of wazp to identify populations in the proportions in which they actually occur.
* The reasonable agreement between DC2 and DP0 results shows that introducing photo-z-related uncertainties in the WaZP cluster detection does not change the overall trend of red fraction evolution.
* As WaZP does not rely on the red sequence information, its good performance in detecting clusters with a low fraction of red galaxies makes the algorithm suitable for studies of cluster properties at high redshifts.

## Future Works:


We have some preliminar code for the investigation of the inclination of the red sequence - the red slope.

In [ ]:
def green_valley(mu1, sigma1, A1, mu2, sigma2, A2):
    g_pos = ((mu1*sigma2**2 - mu2*sigma1**2)+np.sqrt((mu1*sigma2**2 - mu2*sigma1**2)**2 - (sigma2**2-sigma1**2)*(sigma2**2*mu1**2 - sigma1**2*mu2**2 -2*sigma1**2*sigma2**2*np.log(A1/A2))))/(sigma2**2-sigma1**2)
    g_neg = ((mu1*sigma2**2 - mu2*sigma1**2)-np.sqrt((mu1*sigma2**2 - mu2*sigma1**2)**2 - (sigma2**2-sigma1**2)*(sigma2**2*mu1**2 - sigma1**2*mu2**2 -2*sigma1**2*sigma2**2*np.log(A1/A2))))/(sigma2**2-sigma1**2)
    
    return (g_pos, g_neg)

def redseq_fit(mag, color):
    fit_result = stats.linregress(mag, color)
    a = fit_result.slope
    b = fit_result.intercept
    a_err = fit_result.stderr
    xfit = np.arange(10, 30, 1)
    yfit = a*xfit + b
    
    return a, b, a_err, xfit, yfit

wazp_membership_color = wazp_membership[f'mag_true_{mag1}'][mask] - wazp_membership[f'mag_true_{mag2}'][mask]
color = np.array(wazp_membership_color)
mag = np.array(wazp_membership[f'mag_true_{mag2}'][mask])
pmem = np.array(wazp_membership['pmem_nfw2d'][mask])

x = color.reshape(-1, 1)

green_line1, green_line2 = green_valley(mu1, sigma1, A1, mu2, sigma2, A2)
green_line = green_line1

plt.rcParams['font.size'] = 15

plt.figure(figsize=[6,6], dpi=300) 
plt.hist2d(mag, color, bins=70, weights=pmem, cmin=0.5, cmax=None, cmap='viridis')
cbar = pl.colorbar()

red = (color>=green_line)
# first fit
a1, b1, a1_err, xfit1, yfit1 = redseq_fit(mag[red], color[red])
c1 = np.mean(mag)
b2 = green_line - a1*c1
y2 = a1*xfit1 + b2
plt.plot(xfit1, yfit1, 'r--', lw=1.0)
plt.plot(xfit1, y2, 'g-', lw=1.0)

red2 = (color>=((a1*mag)+b2))
# second fit
a, b, a_err, xfit, yfit = redseq_fit(mag[red2], color[red2])
slope.append(a)
slope_err.append(a_err)
plt.plot(xfit, yfit, 'r-', lw=1.5, label=f'red slope {round(a,5)} +- {round(a_err,5)}')
plt.hlines(green_line, 16.5, 24.5, linestyles='dashed', color='green')
    
plt.title(f'Red sequence fit for {round(z1,1)}<z<{round(z2,1)}')
plt.xlabel('mag '+mag2)
plt.ylabel(f'{mag1}-{mag2}')
plt.grid()
plt.legend(loc='lower left')
plt.tight_layout()

## Acknowledgement: 
I would like to thank J. Gschwend, M. Aguena, C. Benoist and L. da Costa for the supervision and LSST Corporation and LIneA for the financial support.